In [1]:
# default_exp backup

# backup
 > API details
 

In [2]:
#! python3
# backupToZip.py
# Copies an entire folder and its contents into
# a zip file whose filename increments.
import os
import sys
import datetime
import time
import inspect
import warnings
import hashlib
import zlib
import zipfile
import pickle
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import OrderedDict
__version__ = "0.1.1"

In [3]:
# Force warnings.warn() to omit the source code line in the message
#formatwarning_orig = warnings.formatwarning
#warnings.formatwarning = lambda message, category, filename, lineno, line=None: \
#    formatwarning_orig(message, category, filename, lineno, line='')

def warning_on_one_line(message, category, filename, lineno, file=None, line=None):
    return ' %s:%s: %s:%s' % (filename, lineno, category.__name__, message)
warnings.formatwarning = warning_on_one_line

In [4]:
verbosity = 0
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
if 'verbosity' in locals().keys() and verbosity > 0:
  print(mlist)
vi = sys.version_info
print("version {0}.{1}.{2} of Python".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    mname = name
    if name.startswith("__"):
        continue
    if hasattr(mod, "__version__"):
        mname = name
        if hasattr(mod, "__path__"):
            mname = os.path.split(mod.__path__[0])[1]
        print("version {1} of {0} as {2} ".format(mname, name, mod.__version__))
    elif hasattr(mod, "__file__") and "site-packages" in mod.__file__:
        print("No __version__ for {0} as {1}".format(mname, name))
print(datetime.datetime.now())
del mod
del name

version 3.9.1 of Python
version zlib of zlib as 1.0 
version np of numpy as 1.19.5 
version pd of pandas as 1.2.0 
2021-01-18 14:03:53.577742


In [5]:
def whoami():
    return sys._getframe(1).f_code.co_name
  
def sha_256(fpath, size=4096):
    m = hashlib.sha256()
    with open(fpath, mode='rb') as fp:
        for chunk in iter(lambda: fp.read(size), b''):
            m.update(chunk)
    return m.hexdigest()

In [6]:
def create_new_zip(infilepath, zipfilepath, 
                   compression=zipfile.ZIP_DEFLATED,
                   compresslevel=zlib.Z_DEFAULT_COMPRESSION,
                   verbosity=0):
  import zipfile
  if verbosity > 1:
    print("creating zipfile {0} from {1} <{2}>".format(infilepath, zipfilepath,
                                                      datetime.datetime.now()))
  zf = zipfile.ZipFile(zipfilepath, mode='w', compression=compression,
                      compresslevel=compresslevel)
  try:
    if verbosity > 1:
      print("adding {0}".format(infilepath))
    zf.write(infilepath)
  finally:
    if verbosity > 1:
      print('Done, closing <{0}>'.format(datetime.datetime.now()))
    zf.close()

In [7]:
def path2string(fpath, sep="_", verbosity=0):
    pathstring = ""
    pathleft = fpath
    while True:
        pathleft, tail = os.path.split(pathleft)
        if len(tail) == 0:
            break
        pathstring = tail + sep + pathstring
    if verbosity > 0:
        print("pathstring= {0}".format(pathstring)) 
    return pathstring
  
  
def check_outdir(outdir, create=True, verbosity=0):
    if os.path.isdir(outdir):
      return True
    
    warnings.warn("{0} not a dir".format(outdir))      
    if not create:
      return False
    
    if verbosity > 0:
      print("trying to create {0}".format(outdir))
    os.makedirs(outdir)
    if not os.path.isdir(outdir):
        raise RuntimeError("Cannot make dir= '{0}'".format(outdir)) 
    return True
    
def make_outfilepath(folder, outdir, basename="generic",
                     sep = "_", ext="",
                     verbosity=0):
    # Figure out the filename this code should used based on 
    # what files already exist.  
    while True:
        nowstr = datetime.datetime.now().strftime(format="%Y-%m-%d__%H_%M_%S") 
        outfilename = basename + sep + nowstr + ext
        if not os.path.exists(outfilename):
            break
        number = number + 1
    if verbosity > 0:
        print("Creating '{0}'".format(outfilename))

    outfilepath = os.path.join(outdir, outfilename)    
    return outfilepath
  
def make_tempfilepath(folder, base, sep="_", ext="", verbosity=0):
    number = 1
    while True:
        nowstr = datetime.datetime.now().strftime(format="%Y-%m-%d__%H_%M_%S")  
        filename = base + sep +  nowstr + ext
        filepath = os.path.join(folder, filename)
        if not os.path.exists(filepath):
            break
        number = number + 1 
    return filepath
  

In [8]:
def backup(folder, outdir, 
           include_exts=None,
                exclude_exts=None,
               meta_fp=None,
                recursive = False,
              comp_thresh = 0.9,
                   compression=zipfile.ZIP_DEFLATED,
                   compresslevel=zlib.Z_DEFAULT_COMPRESSION,       
                testing=False,
                verbosity=0):
    # Backup the entire contents of "folder" into a zip file.
    if verbosity > 0:
        print(whoami())
        print("outdir= {0}".format(outdir))
        print("testing: {0}, recursive: {1}".format(testing, recursive))
    check_outdir(outdir, create=True, verbosity=verbosity)
    
    for xname in ('include_exts', 'exclude_exts'):
        x = locals()[xname]
        if isinstance(x, str):
            x = [x]
        if isinstance(x, list):
            if len(x) == 0:
                x = None
        elif x is not None:
            raise ValueError("{0} should be None or string or list of strings")
        if verbosity > 1:
            print("{0}: {1}".format(xname, x))
        locals()[xname] = x

    folder = os.path.abspath(folder) # make sure folder is absolute

    if not meta_fp:
      outfilepath = make_outfilepath(folder, outdir=outdir,
                                     basename="backup_meta",
                                     ext=".pickle",
                                     verbosity=verbosity)
      meta_fp = open(outfilepath, mode='wb')
      ddict = OrderedDict()
      ddict['rec_type'] = "meta_info"
      ddict['comp_thresh'] = comp_thresh
      ddict['compression'] = compression
      ddict['compresslevel'] = compresslevel
      ddict['backup_version'] = __version__
      ddict['python_version'] = str(sys.version_info)
      ddict['zlib_version'] = zlib.__version__
      ddict["now"] = datetime.datetime.now()
      pickle.dump(ddict, meta_fp)      
      
    # Walk the entire folder tree and compress the files in each folder.

    for dirpath, dirnames, filenames in os.walk(folder, topdown=True):
        if verbosity > 0:
            print("Adding files in '{0}'".format(dirpath))
        # Add the current folder to the ZIP file.
        # recursive call
        if False:
            backup(foldername, include_exts=include_exts,
            exclude_exts=exclude_exts, outpath=outpath,   ofp=ofp,
            testing=testing, recursive=True,  verbosity=verbosity)
        # Add all the files in this folder to the ZIP file.
        for filename in tqdm(filenames):
            base, ext = os.path.splitext(filename)
            if include_exts is not None:
                if ext not in  include_exts:
                    if verbosity > 1:
                        print("  Skipping {0}, {1} not in include_exts".format(filename))
                    continue
            if exclude_exts is not None:
                if ext in exlude_exts:
                    if verbosity > 1:
                        print("  Skipping {0}, {1}  in include_exts".format(filename))
                    continue
            if filename.endswith('.pickle'):
                continue # don't backup the backup pickle files
                
            origfilepath = os.path.join(dirpath, filename)
            if testing and (verbosity > 0):
                print("  adding {0}".format(filename))
            else:
              try:
                ddict = OrderedDict()
                zipfilepath = make_tempfilepath(folder, base="temp", ext=".zip", 
                                                verbosity=verbosity)
                
                create_new_zip(origfilepath, zipfilepath)
                 
                zfile = zipfile.ZipFile(zipfilepath, mode='r')  
                for zm in zfile.infolist():
                  print(zm)
                zfile.close()

                orig_size = os.path.getsize(origfilepath)
                comp_size = os.path.getsize(zipfilepath)
                ddict['rec_type'] = "file_info"
                ddict['filename'] = filename   
                ddict['folder'] = dirpath               
                ddict['filepath'] = origfilepath
                ddict['orig_size'] = orig_size
                ddict['comp_size'] = comp_size   
                ddict['zipname'] = zm.filename               
                ddict['sha256'] = sha_256(origfilepath, size=4096)
                dt_fmt = '%Y-%m-%dT%H:%M:%S'
                ddict['ctime'] = datetime.datetime.fromtimestamp(os.path.getctime(origfilepath)).strftime(dt_fmt)
                ddict['mtime'] = datetime.datetime.fromtimestamp(os.path.getmtime(origfilepath)).strftime(dt_fmt) 
                comp_ratio = np.nan
                if orig_size == 0:
                  warnings.warn("{0} in {1} size is {2}".format(filename, folder, orig_size))

                else:
                  comp_ratio = float(comp_size)/orig_size 
                ddict['comp_ratio'] =  comp_ratio
                
                if ddict['comp_ratio'] > comp_thresh:
                  ddict['compressed'] = False
                  infilepath = origfilepath
                else:
                  infilepath = zipfilepath
                  ddict['compressed'] = True                  

                if verbosity > 0:
                    print("filename: {0}, filepath: {1}".format(filename, origfilepath))
                    print("osize= {0}, csize= {1}".format(orig_size, comp_size))
                    print("compressed= {0}".format(ddict['compressed']))  
                    print("sha_256= {0}".format(ddict['sha256']))
                # write metadata
                pickle.dump(ddict, meta_fp)
                
                # write the file
                outfilepath = os.path.join(outdir, ddict['sha256'])
                shutil.copy(infilepath, outfilepath)    
                
                # remove the temp zipfile
                if os.path.isfile(zipfilepath):
                  os.remove(zipfilepath)
              except Exception as e:
                (extype, exval, tb) = sys.exc_info()
                print("extype= {0}, exval= {1}\n {2}".format(extype, exval, tb))
                raise(Exception(e))

    if not recursive:
        if verbosity > 0:
            print("Done")
        meta_fp.close()

In [9]:
def import_backup_metafile(folder, filename, verbosity=0):
  filepath = os.path.join(folder, filename)
  if not os.path.isfile(filepath):
    raise ValueError("Cannot find file {0} in folder {1}".format(filename, folder))
  data = []
  with open(filepath, "rb") as fp:
      while True:
        try:
          x = pickle.load(fp)      
          data.append(x)
        except EOFError as error:
          # this is expected
          break
        except Exception as e:
          (extype, exval, tb) = sys.exc_info()
          print("extype= {0}, exval= {1}\n {2}".format(extype, exval, tb))
          raise(Exception(e))
  return data

In [10]:
def check_folder_filename(folder, filename):
  filepath = os.path.join(folder, filename)
  if not os.path.isfile(filepath):
    raise ValueError("Cannot find file {0} in folder {1}".format(filename, folder))
  meta = import_backup_metafile(folder=folder, filename=filename)
  if len(meta) == 0:
    warnings.warn("Empty metafile {0} in {1}".format(filename, folder))
    return False  
  return True


def get_meta(folder, filename):
  if not check_folder_filename(folder, filename):
    return False
  
  meta = import_backup_metafile(folder=folder, filename=filename)
  if len(meta) == 0:
    warnings.warn("Empty metafile {0} in {1}".format(filename, folder))
    return None 
  
  if not meta[0]['rec_type'] == "meta_info":
    msg = "file= {0}, folder= {1}\n first elem is not meta {2}".format(filename, folder, meta[0])
    warnings.warn(msg)
    return None
  return meta

def get_meta_fields(folder, filename):
  if not check_folder_filename(folder, filename):
    return False
  
  meta = get_meta(folder, filename)
  if not meta:
    return None
  
  res = {"meta_info": list(meta[0].keys())}
  if len(meta) > 1:
    res["file_info"] = list(meta[1].keys())
  return res
  
  
def get_meta_info(folder, filename, meta_fields=None, 
                    file_info_fields=None, verbosity=0):
  if not check_folder_filename(folder, filename):
    return False
  
  meta = get_meta(folder, filename)
  if not meta:
    return None
  res = ""
  act_fields = get_meta_fields(folder, filename)
  fields = []
  if meta_fields:
    for f in meta_fields:
      if f in act_fields['meta_info']:
        fields.append(f)
      else:
        warnings.warn(" requested meta_field {0} not in meta_fields".format(f))
  else:
    fields = act_fields['meta_info']      

  msglst = ["{0}: {1}".format(f, meta[0][f]) for f in fields]
  res += ", ".join(msglst)
  res += "\n"
  
  nfiles = sum([int(e['rec_type']=='file_info') for e in meta])
  res += "{0} files".format(nfiles)
  res += "\n"
  
  fields = []
  if file_info_fields:
    for f in file_info_fields:
      if f in act_fields['file_info']:
        fields.append(f)
      else:
        warnings.warn(" requested file_info_field {0} not in file_info_fields".format(f))
  else:
    fields = act_fields['file_info'] 
    
  for i, elem in enumerate(meta[1:]):
    msglst = ["[{0}]: {1}: {2}".format(i, f, elem[f]) for f in fields]
    res += ", ".join(msglst)
    res += "\n"
  return res

In [34]:
def recover(folder, meta_filename, filelist, 
            outdir, create_outdir=False,
            chunk_size = 10**6,
            overwrite=False, testing=True, verbosity=0):
  """
  """
  if not os.path.isdir(folder):
    warnings.warn("{0} is not a folder".format(folder))
    return None
  
  meta = get_meta(folder, meta_filename)
  
  if not meta:
    return None  
  if len(meta) ==1:
    warnings.warn("No file_info records")
    return None
  res = check_outdir(outdir, create=create_outdir, verbosity=0)  
  
  filemap = {}
  for i, e in enumerate(meta[1:]):
    filemap[e['filename']] = i+1 
  for filename in filelist:
    if filename in filemap.keys():
      ei = filemap[filename]
      msg = "Found {0} as entry {1}".format(filename, ei)
      print(msg)
      file_info = meta[ei]
      print(file_info)
      if file_info['compressed']:  
        outfilepath = make_tempfilepath(outdir, base="temp", ext=".zip", 
                                                verbosity=verbosity)
      else:
        outfilepath = os.path.join(outdir, file_info['filename'])
      print("outfilepath= {0}".format(outfilepath))
      outfilepath = os.path.abspath(outfilepath) # make sure folder is absolute 
      print("outfilepath= {0}".format(outfilepath))      
      infilename = file_info['sha256']
      infilepath = os.path.join(folder, infilename)
      if not os.path.isfile(infilepath):
        warnings.warn("Cannot fine backup file {0} in {1}".format(infilename, folder))
        continue
      try:
        if verbosity > 0:
          print("copying {0} to {1}".format(infilepath, outfilepath))
        shutil.copy(infilepath, outfilepath)         
      except Exception as e:
        (extype, exval, tb) = sys.exc_info()
        warnings.warn("extype= {0}, exval= {1}\n {2}".format(extype, exval, tb))  

        
      if file_info['compressed']:
        zipfilepath = outfilepath
        outfilepath = os.path.join(outdir, file_info['filename'])
        print("outfilepath {0}".format(outfilepath))
        if verbosity > 0:
          print("Unzipping {0} to {1}".format(zipfilepath, outfilepath))  
          
        zfile = zipfile.ZipFile(zipfilepath, mode='r')  
        for zm in zfile.infolist():
          print(zm)
        try:
          zipname = file_info['zipname']
          print("zipname= {0}  outfilepath= {1}".format(zipname, outfilepath))
          zfile.extract(member=zipname, 
                  path=outfilepath, pwd=None)
        except Exception as e:
          (extype, exval, tb) = sys.exc_info()
          warnings.warn("extype= {0}, exval= {1}\n {2}".format(extype, exval, tb))           
          raise Exception(e)
        zfile.close() 
        os.remove(zipfilepath)
      
      #with open(infilepath, mode='rb') as ifp:
      #  with open(outfilepath, mode="wb") as ofp:
      #    while True:
      #      ifp.read()
    else:
      msg = "No entry for {0}".format(filename)
      warnings.warn(msg)
    return None

In [19]:
if True:
    bpath = os.path.join("C:\\", "Users", "jmull", "OneDrive", "Pictures")
    print("Path= {0}".format(bpath))
    N = 10
    print("files: {0}".format(os.listdir(bpath)[:N]))
    backup(bpath, outdir="backup", testing=False, verbosity=0)

 16%|█████████████▍                                                                     | 5/31 [00:00<00:00, 39.31it/s]

Path= C:\Users\jmull\OneDrive\Pictures
files: ['Camera Roll', 'desktop.ini', 'ga_sos_helper_bot.PNG', 'GeorgeHaroldHaslam.PNG', 'honest-conversation.PNG', 'IMG_0508.JPG', 'IMG_0547-2MP.png', 'IMG_0547-4MP.png', 'IMG_0547-point25MP.png', 'John_ccr_no_tie.jpg']
<ZipInfo filename='Users/jmull/OneDrive/Pictures/desktop.ini' compress_type=deflate filemode='-rw-rw-rw-' file_size=520 compress_size=204>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/ga_sos_helper_bot.PNG' compress_type=deflate filemode='-rw-rw-rw-' file_size=23935 compress_size=20951>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/GeorgeHaroldHaslam.PNG' compress_type=deflate filemode='-rw-rw-rw-' file_size=65904 compress_size=65640>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/honest-conversation.PNG' compress_type=deflate filemode='-rw-rw-rw-' file_size=73641 compress_size=68879>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/IMG_0508.JPG' compress_type=deflate filemode='-rw-rw-rw-' file_size=3191985 compress_size=

 42%|██████████████████████████████████▍                                               | 13/31 [00:00<00:00, 18.22it/s]

<ZipInfo filename='Users/jmull/OneDrive/Pictures/IMG_0547-4MP.png' compress_type=deflate filemode='-rw-rw-rw-' file_size=9212759 compress_size=9215213>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/IMG_0547-point25MP.png' compress_type=deflate filemode='-rw-rw-rw-' file_size=659075 compress_size=659280>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/John_ccr_no_tie.jpg' compress_type=deflate filemode='-rw-rw-rw-' file_size=163159 compress_size=139592>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/john_eye_perscription.jpg' compress_type=deflate filemode='-rw-rw-rw-' file_size=104430 compress_size=58405>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/mail-loop.PNG' compress_type=deflate filemode='-rw-rw-rw-' file_size=38217 compress_size=30094>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/Max_eye_perscription.jpg' compress_type=deflate filemode='-rw-rw-rw-' file_size=103114 compress_size=56996>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/phillips-power-strip.jpg' comp

 <ipython-input-8-c81e33923ca6>:108: UserWarning:temp_2021-01-18_12 in C:\Users\jmull\OneDrive\Pictures size is 0

<ZipInfo filename='Users/jmull/OneDrive/Pictures/pogo.jpg' compress_type=deflate filemode='-rw-rw-rw-' file_size=371535 compress_size=371171>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/taotronics-massager.PNG' compress_type=deflate filemode='-rw-rw-rw-' file_size=11887 compress_size=10486>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/temp_1.zip' compress_type=deflate filemode='-rw-rw-rw-' file_size=384 compress_size=316>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/temp_2.zip' compress_type=deflate filemode='-rw-rw-rw-' file_size=384 compress_size=316>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/temp_2021-01-18_12' compress_type=deflate filemode='-rw-rw-rw-' file_size=0 compress_size=2>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/temp_2021-01-18__12_42_29.zip' compress_type=deflate filemode='-rw-rw-rw-' file_size=196 compress_size=104>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/temp_2021-01-18__12_47_17.zip' compress_type=deflate filemode='-rw-rw-rw-' file

 90%|██████████████████████████████████████████████████████████████████████████        | 28/31 [00:04<00:00,  5.05it/s]

<ZipInfo filename='Users/jmull/OneDrive/Pictures/TheOneAboutVoting.MOV' compress_type=deflate filemode='-rw-rw-rw-' file_size=136045619 compress_size=127206391>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/TheOtherOneAboutVoting-jhm.MOV' compress_type=deflate filemode='-rw-rw-rw-' file_size=69076081 compress_size=64846562>


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 69.15it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.18it/s]

<ZipInfo filename='Users/jmull/OneDrive/Pictures/Walter-Erdos.PNG' compress_type=deflate filemode='-rw-rw-rw-' file_size=74907 compress_size=65362>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/Camera Roll/desktop.ini' compress_type=deflate filemode='-rw-rw-rw-' file_size=190 compress_size=133>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/Camera Roll/WIN_20210113_14_04_10_Pro.jpg' compress_type=deflate filemode='-rw-rw-rw-' file_size=340130 compress_size=321523>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/Saved Pictures/desktop.ini' compress_type=deflate filemode='-rw-rw-rw-' file_size=190 compress_size=132>
<ZipInfo filename='Users/jmull/OneDrive/Pictures/Screenshots/desktop.ini' compress_type=deflate filemode='-rw-rw-rw-' file_size=190 compress_size=132>


In [20]:
meta_folder = "backup"
files = os.listdir(meta_folder)
meta_files = [f for f in files if f.endswith("pickle")]
meta_files

['backup_meta_2021-01-18__14_22_14.pickle']

In [27]:
meta_folder = "backup"
files = os.listdir(meta_folder)
meta_files = [f for f in files if f.endswith("pickle")]
meta_filename = meta_files[0]
print(meta_filename)
#print_meta_info(folder=meta_folder, filename=meta_filename, fields=[''])
meta_fields = get_meta_fields(meta_folder, meta_filename)
meta_fields
res = get_meta_info(folder=meta_folder, filename=meta_filename, meta_fields=None, 
              file_info_fields=['filename', 'zipname', 'orig_size'])
print(res)

backup_meta_2021-01-18__14_22_14.pickle
rec_type: meta_info, comp_thresh: 0.9, compression: 8, compresslevel: -1, backup_version: 0.1.1, python_version: sys.version_info(major=3, minor=9, micro=1, releaselevel='final', serial=0), zlib_version: 1.0, now: 2021-01-18 14:22:14.483168
35 files
[0]: filename: desktop.ini, [0]: zipname: Users/jmull/OneDrive/Pictures/desktop.ini, [0]: orig_size: 520
[1]: filename: ga_sos_helper_bot.PNG, [1]: zipname: Users/jmull/OneDrive/Pictures/ga_sos_helper_bot.PNG, [1]: orig_size: 23935
[2]: filename: GeorgeHaroldHaslam.PNG, [2]: zipname: Users/jmull/OneDrive/Pictures/GeorgeHaroldHaslam.PNG, [2]: orig_size: 65904
[3]: filename: honest-conversation.PNG, [3]: zipname: Users/jmull/OneDrive/Pictures/honest-conversation.PNG, [3]: orig_size: 73641
[4]: filename: IMG_0508.JPG, [4]: zipname: Users/jmull/OneDrive/Pictures/IMG_0508.JPG, [4]: orig_size: 3191985
[5]: filename: IMG_0547-2MP.png, [5]: zipname: Users/jmull/OneDrive/Pictures/IMG_0547-2MP.png, [5]: orig_si

In [35]:
recover(folder=meta_folder, meta_filename=meta_filename,
        filelist=['mail-loop.PNG'], 
            outdir='recovered', create_outdir=True,
            overwrite=True, testing=False, verbosity=1)


Found mail-loop.PNG as entry 11
OrderedDict([('rec_type', 'file_info'), ('filename', 'mail-loop.PNG'), ('folder', 'C:\\Users\\jmull\\OneDrive\\Pictures'), ('filepath', 'C:\\Users\\jmull\\OneDrive\\Pictures\\mail-loop.PNG'), ('orig_size', 38217), ('comp_size', 30278), ('zipname', 'Users/jmull/OneDrive/Pictures/mail-loop.PNG'), ('sha256', '39ae971965c14e3c69b9a03cac415d1d6f47960c625090098d7d21650ab3c9a1'), ('ctime', '2021-01-08T10:11:42'), ('mtime', '2021-01-08T10:11:43'), ('comp_ratio', 0.7922652222832771), ('compressed', True)])
outfilepath= recovered\temp_2021-01-18__14_38_11.zip
outfilepath= C:\Users\jmull\Documents\Github\py-backup\recovered\temp_2021-01-18__14_38_11.zip
copying backup\39ae971965c14e3c69b9a03cac415d1d6f47960c625090098d7d21650ab3c9a1 to C:\Users\jmull\Documents\Github\py-backup\recovered\temp_2021-01-18__14_38_11.zip
outfilepath recovered\mail-loop.PNG
Unzipping C:\Users\jmull\Documents\Github\py-backup\recovered\temp_2021-01-18__14_38_11.zip to recovered\mail-loop.P

In [ ]:
meta_filename = "backup_meta_1.pickle"
meta_folder = "backup"
meta = import_backup_metafile(folder=meta_folder, filename=meta_filename)
meta[3]